In [ ]:
import keras, numpy as np, matplotlib as mpl, matplotlib.pyplot as plt, tqdm
import os.path
import time
%matplotlib inline

In [ ]:
import nnUtils

In [ ]:
import sklearn, sklearn.model_selection

In [ ]:
datadir = os.path.join(os.path.expanduser('~'), 'data2', 'behavioralCloning')

In [ ]:
# data = np.load(os.path.join(datadir, 'multiData-mouseForwardFuller.npz'))
# X = data['X']
# Y = data['Y'][:, 0].reshape(-1, 1)

In [ ]:
!ls ~/data2/behavioralCloning/*.zip

In [ ]:
import loadData

from importlib import reload
reload(loadData);

In [ ]:
dataGenerator = loadData.DataGenerator(
    [
        os.path.join(loadData.HOME, 'data2', 'behavioralCloning', p)
        for p in (
            'mouseForward.zip',
            'mouseReverse.zip',
            'dirtSidesForward.zip',
            'dirtSidesReverse.zip',
        )
    ],
    MAXDATA=2000,
    responseKeys=('steering',),# 'throttle', 'brake',)
    #verbose=False,
)

In [ ]:
modelname = 'inceptionv3_deeper_wider-mfwd_sidecams-fullgen-junglecenter-tanh-normalInput'

In [ ]:
def prod(it):
    out = 1
    for x in it:
        try:
            out *= float(x)
        except TypeError:
            out *= int(x)
    return out

In [ ]:
%%time
def VGG(
    nout=1, input_shape=dataGenerator.sampleImageShape, doCompile=True, 
    loss='mse', optimizer='nadam', metrics=['accuracy', 'mae'],
    ):
    
    # Normalize.
    x = keras.layers.Input(input_shape)
    x = keras.layers.Lambda(lambda y: (y / 255.0) - 0.5)(x)
    
    model = keras.applications.inception_v3.InceptionV3(
        include_top=False, 
        weights='imagenet', 
        input_tensor=x,
        #input_shape=input_shape, 
        #pooling=None, 
        #classes=1000
    )
    img_input = model.layers[0].input
    nUnfrozen = sum([prod(w.shape) for w in model.trainable_weights])
    for layer in model.layers:
        layer.trainable = False
    
    # Get flattened output.
    x = model.layers[-1].output
    print('Pretrained model provides {:,} features.'.format(prod(x.shape[1:])))
    x = keras.layers.Flatten()(x)
    
    
    # Add layers.
    init = dict(bias_initializer='zeros', kernel_initializer='glorot_normal')
    x = keras.layers.Dense(16, activation='relu', **init)(x)
#     for k in range(3):
#         x = keras.layers.Dense(16, activation='relu', **init)(x)
#     x = keras.layers.Dense(16, activation='tanh', name='tanh')(x)
#     x = keras.layers.Dense(nout, activation='linear', name='predictions')(x)
    for k in range(4):
        x = keras.layers.Dense(16, activation='relu', **init)(x)
    x = keras.layers.Dense(nout, activation='tanh', name='tanh')(x)
    
    model = keras.Model(inputs=img_input, outputs=x)
    
    nFrozen = sum([prod(w.shape) for w in model.trainable_weights])
    print("Freezing reduces trainable size from {:,} to {:,} parameters.".format(nUnfrozen, nFrozen))
    
    if doCompile:
        model.compile(
            loss=loss,
            optimizer=optimizer,
            metrics=metrics,
        )
    return model

model = VGG(
    optimizer=keras.optimizers.Nadam(
        lr=0.0001, #beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    )
)


In [ ]:
from nnUtils import fitModelWithDataGenerator

In [ ]:
NEPOCH = 32

In [ ]:
trainGen = dataGenerator.generate()
print(len(trainGen))
validGen = dataGenerator.generate(validation=True)

In [ ]:
# def trainYielder():
#     while True:
#         yield trainGen.__next__()

# for x in trainYielder():
#     print([y.shape for y in x])

In [ ]:
%%time
history = nnUtils.fitModelWithDataGenerator(model, dataGenerator, modelname, epochs=NEPOCH)

In [ ]:
%%time
fpath = os.path.join(datadir, '%s.h5' % modelname)
print('Saving to', fpath)
model.save(fpath)

In [ ]:
!recent /home/tsbertalan/data2/behavioralCloning/ | head -n 4

In [ ]:
imgs = []
responses = []
for k in range(1000):
    image, response = dataGenerator.getImageResponse(k)
    imgs.append(image.reshape((1, *image.shape)))
    responses.append(response)
X = np.vstack(imgs)
Y = np.vstack(responses)

In [ ]:
pred = model.predict(X)
pred.shape, Y.shape

In [ ]:
from collections import deque
def runningMean(x, N):
    y = []
    hist = deque(maxlen=N)
    for a in x:
        hist.append(a)
        y.append(np.mean(hist))
    return np.array(y)

In [ ]:
fig, ax = plt.subplots()
start = 100
end = start + 100
ax.plot(Y[start:end], label=r'truth $\theta$')
ax.plot(pred[start:end], label=r'predictions $\hat\theta$')
filtersize = 3
scale = 1
smoothed = scale * runningMean(pred, filtersize)
ax.plot(
    smoothed[start:end], 
    label=r'$\hat\rho = %.1f \cdot box_{%d}(\hat\theta)$' % (scale, filtersize)
)
ax.set_xlabel('time [samples]')
ax.set_ylabel('angle [radians]')
ax.legend();
fig.savefig('smoothingEffect.png')